**Online Retailer Data Analysis - Part 9 - A/B Testing**

This notebook is the last of nine parts in a project designed to demonstrate various data analytics and data science techniques, highlighting how they can enhance business intelligence and drive effective decision-making.

Using the same dataset from the previous notebook, this project will cover the following topics:

- Calculating Metrics
- Customer Segmentation
- Customer Lifetime Value Prediction
- Churn Prediction
- Predicting Next Purchase Day
- Predicting Sales
- Market Response Models
- Uplift Modeling
- **A/B Testing Design and Execution**

In [47]:
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division #must if you use python 2
from sklearn.cluster import KMeans


import sklearn
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

1- What is our hypothesis?

Moving forward our hypothesis is; test group will have more retention:

**Group A → Offer → Higher Retention**

**Group B → No offer → Lower Retention**

This also helps us to test model accuracy as well. If group B’s retention rate is 50%, it clearly shows that our model is not working. The same applies to measuring revenue coming from those users too.

2- What is our success metric?

In this case, we are going to check the retention rate of both groups.

Programmatic A/B Testing
For this coding example, we are going to create our own dataset by using numpy library and evaluate the result of an A/B test.

The dataset will contain the columns below:

- customer_id: the unique identifier of the customer
- segment: customer’s segment; high-value or low-value
- group: indicates whether the customer is in the test or control group
- purchase_count: # of purchases completed by the customer

In [48]:
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range (20000)])
df_hv['segment'] = np.array(['high-value' for _ in range(20000)])
df_hv['group'] = 'control'
df_hv.loc[df_hv.index<10000, 'group'] = 'test'

df_hv.head()

,customer_id,segment,group
0,0,high-value,test
1,1,high-value,test
2,2,high-value,test
3,3,high-value,test
4,4,high-value,test


In [49]:
df_hv.group.value_counts()

,count
group,
test,10000
control,10000


Ideally, purchase count should be a **Poisson distribution**. There will be customers with no purchase and we will have less customers with high purchase counts. Let’s use numpy.random.poisson() for doing that and assign different distributions to test and control group:

In [50]:
df_hv.loc[df_hv.group == 'test', 'purchase_count'] = np.random.poisson(0.6, 10000) #0.6 is the mean number of purchases per customer
df_hv.loc[df_hv.group == 'control', 'purchase_count'] = np.random.poisson(0.5, 10000)

In [51]:
df_hv.head()

,customer_id,segment,group,purchase_count
0,0,high-value,test,0.0
1,1,high-value,test,0.0
2,2,high-value,test,0.0
3,3,high-value,test,1.0
4,4,high-value,test,0.0


In [52]:
df_hv.nunique()

,0
customer_id,20000
segment,1
group,2
purchase_count,6


We have everything to evaluate our A/B test. Assume we applied an offer to 50% of high-value users and observed their purchases in a given period. Best way to visualize it to check the densities:

In [5]:
test_results = df_hv[df_hv.group == 'test'].purchase_count
control_results = df_hv[df_hv.group == 'control'].purchase_count

hist_data = [test_results, control_results]

group_labels = ['test', 'control']

In [6]:

import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [7]:
# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, bin_size=.5,
                         curve_type='normal',show_rug=False)

fig.layout = go.Layout(
        title='High Value Customers Test vs Control',
        plot_bgcolor  = 'rgb(243,243,243)',
        paper_bgcolor  = 'rgb(243,243,243)',
    )


# Plot!
pyoff.iplot(fig)

The results are looking really good. The density of the test group’s purchase is better starting from 1. But how can we with certainty say this experiment is successful and the difference didn’t happen due to other factors?

To answer this question, we need to check if the uptick in the test group is statistically significant. scipy library allows us to programmatically check this:

In [8]:
from scipy import stats
test_result = stats.ttest_ind(test_results, control_results)
print(test_result)

TtestResult(statistic=10.015821872324246, pvalue=1.4759489984569612e-23, df=19998.0)


Here’s a detailed interpretation of each component:

1. T-Test Statistic: 11.311609975232374
Magnitude: The t-test statistic of 11.3116 is very large, indicating a substantial difference between the sample means (or between a sample mean and a hypothesized population mean). The larger the t-statistic, the greater the effect size or difference.
2. P-Value: 1.4139613159645588e-29
Significance: The p-value is 1.4139613159645588e-29, which is extremely small. In practical terms, this is much smaller than any commonly used significance level (e.g., 0.05, 0.01, 0.001). A p-value this small suggests that the observed difference is highly unlikely to have occurred by random chance alone.

Conclusion: Given the very small p-value, you can reject the null hypothesis with very high confidence. The null hypothesis typically states that there is no effect or no difference. The tiny p-value indicates strong evidence against this hypothesis, suggesting that the effect observed is statistically significant.

3. Degrees of Freedom (df): 19998.0
Context: Degrees of freedom in a t-test typically depend on the sample sizes involved. In many t-tests, the degrees of freedom are calculated as the total number of observations minus the number of groups or conditions (for independent t-tests) or related to the sample size and number of groups (for paired t-tests).

Implications: With 19998.0 degrees of freedom, this is a very large sample size. Large degrees of freedom generally provide a more accurate estimate of the t-distribution, making the results more reliable. It also means the critical values for significance are relatively small, making it easier to achieve a significant result.

To understand if our test is statistically significant or not, let’s build a function and apply to our dataset:

In [9]:
def eval_test(test_results,control_results):
    test_result = stats.ttest_ind(test_results, control_results)
    if test_result[1] < 0.05:
        print('result is significant')
    else:
        print('result is not significant')

In [10]:
eval_test(test_results, control_results)

result is significant


Looks great but unfortunately, it is not that simple. If you select a biased test group, your results will be statistically significant by default. As an example, if we allocate more high-value customer to test group and more low-value customers to control group, then our experiment becomes a failure from the beginning. That’s why selecting the group is the key to a healthy A/B test.

**Selecting Test & Control Groups**

The most common approach to select test & control groups is random sampling. Let’s see how we can do it programmatically. We are going to start with creating the dataset first. In this version, it will have 20k high-value and 80k low-value customers:

In [11]:
#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(20000)])
df_hv['segment'] = np.array(['high-value' for _ in range(20000)])
df_hv['prev_purchase_count'] = np.random.poisson(0.9, 20000)


df_lv = pd.DataFrame()
df_lv['customer_id'] = np.array([count for count in range(20000,100000)])
df_lv['segment'] = np.array(['low-value' for _ in range(80000)])
df_lv['prev_purchase_count'] = np.random.poisson(0.3, 80000)
df_customers = pd.concat([df_hv,df_lv],axis=0)

In [12]:
df_customers.head()

,customer_id,segment,prev_purchase_count
0,0,high-value,0
1,1,high-value,2
2,2,high-value,0
3,3,high-value,1
4,4,high-value,1


In [13]:
df_customers.tail()

,customer_id,segment,prev_purchase_count
79995,99995,low-value,0
79996,99996,low-value,0
79997,99997,low-value,2
79998,99998,low-value,0
79999,99999,low-value,1


By using pandas’ sample() function, we can select our test groups. Assuming we will have 90% test and 10% control group:

In [14]:
df_test = df_customers.sample(frac=0.9)
df_control = df_customers[~df_customers.customer_id.isin(df_test.customer_id)]

df_test: Contains a random 90% sample of df_customers.

df_control: Contains the remaining 10% of df_customers, i.e., those rows not included in df_test.

In this example, we extracted 90% of the whole group and labeled it as test. But there is a small problem that can ruin our experiment. If you have significantly different multiple groups in your dataset (in this case, high-value & low-value), better to do random sampling separately. Otherwise, we can’t guarantee that the ratio of high-value to low-value is the same for test and control group.

To ensure creating test and control groups correctly, we need to apply the following code:

In [53]:
df_test_hv = df_customers[df_customers.segment == 'high-value'].sample(frac=0.9)
df_test_lv = df_customers[df_customers.segment == 'low-value'].sample(frac=0.9)
df_test = pd.concat([df_test_hv,df_test_lv],axis=0)
df_control = df_customers[~df_customers.customer_id.isin(df_test.customer_id)]

In [16]:
df_test.segment.value_counts()

,count
segment,
low-value,72000
high-value,18000


In [17]:
df_control.segment.value_counts()

,count
segment,
low-value,8000
high-value,2000


Final Test and Control Groups:

Final df_test: The combined test group, which is now comprised of 90% high-value and 90% low-value customers.

We have explored how to do the t-test and selecting test and control groups. But what if we are doing A/B/C test or A/B test on multiple groups like above. It’s time to introduce ANOVA tests.

**One-way ANOVA**

Let’s assume we are testing 2+ variants on same groups (i.e 2 different offers and no-offer to low-value high-value customers). Then we need to apply one-way ANOVA for evaluating our experiment. Let’s start from creating our dataset:

In [21]:
#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(30000)])
df_hv['segment'] = np.array(['high-value' for _ in range(30000)])
df_hv['group'] = 'A'
df_hv.loc[df_hv.index>=10000,'group'] = 'B'
df_hv.loc[df_hv.index>=20000,'group'] = 'C'

In [22]:

df_hv.group.value_counts()

,count
group,
A,10000
B,10000
C,10000


In [23]:
df_hv.loc[df_hv.group == 'A', 'purchase_count'] = np.random.poisson(0.4, 10000)
df_hv.loc[df_hv.group == 'B', 'purchase_count'] = np.random.poisson(0.6, 10000)
df_hv.loc[df_hv.group == 'C', 'purchase_count'] = np.random.poisson(0.2, 10000)


In [24]:
a_stats = df_hv[df_hv.group=='A'].purchase_count
b_stats = df_hv[df_hv.group=='B'].purchase_count
c_stats = df_hv[df_hv.group=='C'].purchase_count

hist_data = [a_stats, b_stats, c_stats]

group_labels = ['A', 'B','C']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, bin_size=.5,
                         curve_type='normal',show_rug=False)

fig.layout = go.Layout(
        title='Test vs Control Stats',
        plot_bgcolor  = 'rgb(243,243,243)',
        paper_bgcolor  = 'rgb(243,243,243)',
    )


# Plot!
pyoff.iplot(fig)

To evaluate the result, we will apply the function below:

In [25]:
def one_anova_test(a_stats,b_stats,c_stats):
    test_result = stats.f_oneway(a_stats, b_stats, c_stats)
    if test_result[1] < 0.05:
        print('result is significant')
    else:
        print('result is not significant')

The logic is similar to t_test. If p-value is lower than 5%, our test become significant:

In [26]:
one_anova_test(a_stats,b_stats,c_stats)

result is significant


Let’s check what it would look like if there was no difference between the groups:

In [27]:
df_hv.loc[df_hv.group == 'A', 'purchase_count'] = np.random.poisson(0.5, 10000)
df_hv.loc[df_hv.group == 'B', 'purchase_count'] = np.random.poisson(0.5, 10000)
df_hv.loc[df_hv.group == 'C', 'purchase_count'] = np.random.poisson(0.5, 10000)
a_stats = df_hv[df_hv.group=='A'].purchase_count
b_stats = df_hv[df_hv.group=='B'].purchase_count
c_stats = df_hv[df_hv.group=='C'].purchase_count
hist_data = [a_stats, b_stats, c_stats]
group_labels = ['A', 'B','C']
# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, bin_size=.5,
                         curve_type='normal',show_rug=False)
fig.layout = go.Layout(
        title='Test vs Control Stats',
        plot_bgcolor  = 'rgb(243,243,243)',
        paper_bgcolor  = 'rgb(243,243,243)',
    )
# Plot!
pyoff.iplot(fig)

To see if there is a difference between A and B or C, we apply the Two-way ANOVA

**Two-way ANOVA**

Let’s say we are doing the same test on both high-value and low-value customers. In this case, we need to apply two-way ANOVA. We are going to create our dataset again and build our evaluation method:

In [28]:
#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(20000)])
df_hv['segment'] = np.array(['high-value' for _ in range(20000)])
df_hv['group'] = 'control'
df_hv.loc[df_hv.index<10000,'group'] = 'test'
df_hv.loc[df_hv.group == 'control', 'purchase_count'] = np.random.poisson(0.6, 10000)
df_hv.loc[df_hv.group == 'test', 'purchase_count'] = np.random.poisson(0.8, 10000)


df_lv = pd.DataFrame()
df_lv['customer_id'] = np.array([count for count in range(20000,100000)])
df_lv['segment'] = np.array(['low-value' for _ in range(80000)])
df_lv['group'] = 'control'
df_lv.loc[df_lv.index<40000,'group'] = 'test'
df_lv.loc[df_lv.group == 'control', 'purchase_count'] = np.random.poisson(0.2, 40000)
df_lv.loc[df_lv.group == 'test', 'purchase_count'] = np.random.poisson(0.3, 40000)

df_customers = pd.concat([df_hv,df_lv],axis=0)

In [29]:
df_customers.head()

,customer_id,segment,group,purchase_count
0,0,high-value,test,0.0
1,1,high-value,test,1.0
2,2,high-value,test,2.0
3,3,high-value,test,0.0
4,4,high-value,test,0.0


Two-way ANOVA requires building a model like below:

In [54]:
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
model = smf.ols(formula='purchase_count ~ segment + group ', data=df_customers).fit()
# purchase_count is the dependent variable and segment & group are the predictor variables

aov_table = anova_lm(model, typ=2)

In [55]:
print(np.round(aov_table,4))

              sum_sq       df          F  PR(>F)
segment    3256.7616      1.0  9607.2634     0.0
group       329.4760      1.0   971.9356     0.0
Residual  33897.9351  99997.0        NaN     NaN


Interpretation

**Segment:**

- sum_sq (Sum of Squares): 3256.7616
This value represents the amount of variation in purchase_count that is explained by the segment predictor.

- df (Degrees of Freedom): 1.0
Since segment is a categorical variable with two levels (high-value and low-value), it has 1 degree of freedom (number of levels - 1).

- F (F-statistic): 9607.2634
This is the ratio of the mean square of segment to the mean square of the residual. A very high F-statistic indicates that the variability explained by segment is much larger compared to the residual variability, suggesting a strong effect.

-PR(>F) (p-value): 0.0
The p-value is very close to zero, indicating that the effect of segment on purchase_count is statistically significant. In other words, the difference in purchase_count between the high-value and low-value segments is highly significant.

**Group**

- sum_sq (Sum of Squares): 329.4760
This value represents the amount of variation in purchase_count that is explained by the group predictor.

- df (Degrees of Freedom): 1.0
Similar to segment, group has 1 degree of freedom.

- F (F-statistic): 971.9356
The F-statistic for group is also very high, indicating that the variability explained by group is substantial compared to the residual variability.

- PR(>F) (p-value): 0.0
The p-value is again very close to zero, showing that the effect of group on purchase_count is statistically significant.

**Residual**

- sum_sq (Sum of Squares): 33897.9351
This represents the variation in purchase_count that is not explained by the predictors segment and group.

- df (Degrees of Freedom): 99997.0
The residual degrees of freedom is the total number of observations minus the number of predictors minus one (in this case, 100000 - 2 = 99998, which aligns with the output here, likely a small discrepancy due to rounding).

- F and PR(>F): Not applicable (NaN)
The residual row doesn’t have an F-statistic or p-value because it represents the unexplained variation, not a test statistic.

Segment: Both segment and group have highly significant effects on purchase_count, as evidenced by their extremely low p-values (0.0). The segment variable explains a large amount of variability in purchase_count compared to the residual variability, indicating that customers' segments (high-value vs. low-value) are a strong predictor of their purchase counts.


Group: Similarly, the group variable (test vs. control) also has a significant effect, though the amount of variability it explains (sum_sq) is smaller compared to segment. This suggests that while group has a significant effect, segment might be a stronger predictor in this dataset.

Now we know how to select our groups and evaluate the results. But there is one more missing part. To reach statistical significance, our sample size should be enough. Let’s see how we can calculate it.

Sample Size Calculation
To calculate the required sample size, first we need to understand two concepts:

Effect size: this represents the magnitude of difference between averages of test and control group. It is the variance in averages between test and control groups divided by the standard deviation of the control.
Power: this refers to the probability of finding a statistical significance in your test. To calculate the sample size, 0.8 is the common value that is being used.

Let’s build our dataset and see the sample size calculation in an example:

In [56]:
from statsmodels.stats import power
ss_analysis = power.TTestIndPower()


#create hv segment
df_hv = pd.DataFrame()
df_hv['customer_id'] = np.array([count for count in range(20000)])
df_hv['segment'] = np.array(['high-value' for _ in range(20000)])
df_hv['prev_purchase_count'] = np.random.poisson(0.7, 20000)


In [57]:
df_hv.head()

,customer_id,segment,prev_purchase_count
0,0,high-value,0
1,1,high-value,0
2,2,high-value,0
3,3,high-value,1
4,4,high-value,1


In [58]:
purchase_mean = df_hv.prev_purchase_count.mean()
purchase_std = df_hv.prev_purchase_count.std()

In [38]:
print(np.round(purchase_mean,4),np.round(purchase_std,4))

0.7062 0.8447


In this example, the average of purchases (purchase_mean) is 0.7 and the standard deviation (purchase_std) is 0.84.

Let’s say we want to increase the purchase_mean to 0.75 in this experiment. We can calculate the effect size like below:

In [35]:
effect_size = (0.75 - purchase_mean)/purchase_std

After that, the sample size calculation is quite simple:

In [36]:
alpha = 0.05
power = 0.8
ratio = 1
ss_result = ss_analysis.solve_power(effect_size=effect_size, power=power,alpha=alpha, ratio=ratio , nobs1=None)
print(ss_result)

5839.347269082281


Alpha is the threshold for statistical significance (5%) and our ratio of test and control sample sizes are 1 (equal). As a result, our required sample size is (output of ss_result) 4868.

Let’s build a function to use this everywhere we want:

In [39]:
def calculate_sample_size(c_data, column_name, target,ratio):
    value_mean = c_data[column_name].mean()
    value_std = c_data[column_name].std()

    value_target = value_mean * target

    effect_size = (value_target - value_mean)/value_std

    power = 0.8
    alpha = 0.05
    ss_result = ss_analysis.solve_power(effect_size=effect_size, power=power,alpha=alpha, ratio=ratio , nobs1=None)
    print(int(ss_result))

To this function, we need to provide our dataset, the column_name that represents the value (purchase_count in our case), our target mean (0.75 was our target in the previous example) and the ratio.

In the dataset above, let’s assume we want to increase purchase count mean by 5% and we will keep the sizes of both groups the same:

In [40]:
calculate_sample_size(df_hv, 'prev_purchase_count', 1.05,1)

8984


Then the result becomes 8961.